In [1]:
import numpy as np

# Define the Node class for the decision tree
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature # Index of feature to split on
        self.threshold = threshold # Threshold value for splitting
        self.left = left # Left subtree
        self.right = right # Right subtree
        self.value = value # Value at leaf node (predicted fruit)

# Define the CART algorithm class
class CART:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    # Fit the decision tree on the given dataset
    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)

    # Recursive function to build the decision tree
    def _build_tree(self, X, y, depth):
        if depth == self.max_depth or len(set(y)) == 1:
            return Node(value=max(set(y.tolist()), key=y.tolist().count)) # Convert y to list
        
        n_samples, n_features = X.shape
        best_gini = float('inf')
        best_feature, best_threshold = None, None
        
        for feature in range(n_features):
            thresholds = set(X[:, feature])
            for threshold in thresholds:
                left_indices = [i for i, val in enumerate(X[:, feature]) if val <= threshold]
                right_indices = [i for i, val in enumerate(X[:, feature]) if val > threshold]
                
                left_gini = self._gini_impurity(y[left_indices])
                right_gini = self._gini_impurity(y[right_indices])
                
                weighted_gini = (len(left_indices) * left_gini + len(right_indices) * right_gini) / n_samples
                
                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_feature = feature
                    best_threshold = threshold
        
        left_indices = [i for i, val in enumerate(X[:, best_feature]) if val <= best_threshold]
        right_indices = [i for i, val in enumerate(X[:, best_feature]) if val > best_threshold]
        
        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)
        
        return Node(feature=best_feature, threshold=best_threshold, left=left_subtree, right=right_subtree)

    # Calculate the Gini impurity for a given set of labels
    def _gini_impurity(self, y):
        classes = set(y.tolist()) # Convert y to list
        impurity = 1
        for cls in classes:
            p = sum([1 for label in y if label == cls]) / len(y)
            impurity -= p ** 2
        return impurity

    # Predict the fruit for a given input (e.g., color)
    def predict(self, X):
        return [self._traverse_tree(x, self.root) for x in X]

    # Traverse the decision tree to predict the fruit
    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

# Example usage:
if __name__ == "__main__":
    # Toy dataset
    # Features: Color (0: Red, 1: Green), Size (0: Small, 1: Medium, 2: Large)
    X_train = np.array([[0, 0], [1, 1], [0, 1], [1, 2], [1, 0]])
    y_train = np.array(['Apple', 'Orange', 'Apple', 'Orange', 'Apple']) # Convert y_train to numpy array
    
    # Create and train the CART classifier
    cart = CART(max_depth=2)
    cart.fit(X_train, y_train)
    
    # New input: Predict fruit for color (0: Red) and size (1: Medium)
    new_input = np.array([[0, 1]])
    predicted_fruit = cart.predict(new_input)
    print("Predicted Fruit:", predicted_fruit)


Predicted Fruit: ['Apple']
